# Measure qubits

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.2.3
qiskit-ibm-runtime~=0.43.1
```
</details>

To get information about a qubit's state, you can _measure_ it onto a [classical bit](/docs/api/qiskit/circuit#qiskit.circuit.Clbit). In Qiskit, measurements are performed in the computational basis, that is, the single-qubit Pauli-$Z$ basis. Therefore, a measurement yields 0 or 1, depending on the overlap with the Pauli-$Z$ eigenstates $|0\rangle$ and $|1\rangle$:

$$
|q\rangle \xrightarrow{measure}\begin{cases}
      0 (\text{outcome}+1), \text{with probability } p_0=|\langle q|0\rangle|^{2}\text{,} \\
      1 (\text{outcome}-1), \text{with probability } p_1=|\langle q|1\rangle|^{2}\text{.}
    \end{cases}
$$

## Mid-circuit measurements

Mid-circuit measurements are a key component of dynamic circuits. Prior to `qiskit-ibm-runtime` v0.43.0, `measure` was the only measurement instruction in Qiskit. Mid-circuit measurements, however, have different tuning requirements than _terminal_ measurements (measurements that happen at the end of a circuit). For example, you need to consider the instruction duration when tuning a mid-circuit measurement because longer instructions cause noisier circuits. You don't need to consider instruction duration for terminal measurements because there are no instructions after terminal measurements.


In `qiskit-ibm-runtime` v0.43.0, the `MidCircuitMeasure` instruction was introduced. As the name suggests, it is a new measurement instruction that is optimized for mid-circuit on IBM&reg; QPUs.

<Admonition type="note">
The `MidCircuitMeasure` instruction maps to the `measure_2` instruction reported in the backend's `supported_instructions`. However,  `measure_2` is not supported on all backends. Use `service.backends(filters=lambda b: "measure_2" in b.supported_instructions)` to find backends that support it.  New measurements might be added in the future, but this is not guarenteed.
</Admonition>

## Apply a measurement to a circuit

There are several ways to apply measurements to a circuit:

### `QuantumCircuit.measure` method

Use the [`measure`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#measure) method to measure a [`QuantumCircuit`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class).

Examples:

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.x(0)
qc.x(1)
qc.x(4)
qc.measure(
    range(5), range(5)
)  #  Measures all qubits into the corresponding clbit.

In [2]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 1)
qc.x([0, 2])
qc.measure(1, 0)  # Measure qubit 1 into the classical bit 0.

### `Measure` class

The Qiskit [Measure](/docs/api/qiskit/circuit#qiskit.circuit.Measure) class measures the specified qubits.

In [3]:
from qiskit.circuit import Measure

qc = QuantumCircuit(3, 1)
qc.x([0, 1])
qc.append(Measure(), [0], [0])  # measure qubit 0 into clbit 0

### `QuantumCircuit.measure_all` method

To measure all qubits into the corresponding classical bits, use the [`measure_all`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) method. By default, this method adds new classical bits in a `ClassicalRegister` to store these measurements.

In [4]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 1)
qc.x([0, 2])
qc.measure_all()  # Measure all qubits.

### `QuantumCircuit.measure_active` method

To measure all qubits that are not idle, use the [`measure_active`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#measure_active) method. This method creates a new `ClassicalRegister` with a size equal to the number of non-idle qubits being measured.

In [5]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3, 1)
qc.x([0, 2])
qc.measure_active()  # Measure qubits that are not idle, that is, qubits 0 and 2.

<span id="midcircuit"></span>
### `MidCircuitMeasure` method


Use `MidCircuitMeasure` to apply a mid-circuit measurement (requires `qiskit-ibm-runtime` v0.43.0 or later).  While you can use `QuantumCircuit.measure` for a mid-circuit measurement, because of its design, `MidCircuitMeasure` is typically a better choice.  For example, it adds less overhead to your circuit than when using `QuantumCircuit.measure`.

In [6]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.circuit import MidCircuitMeasure
from qiskit.circuit import Measure

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

circ = QuantumCircuit(2, 2)
circ.x(0)
circ.append(MidCircuitMeasure(), [0], [0])
# circ.measure([0], [0])
# circ.measure_all()
print(circ.draw(cregbundle=False))

     ┌───┐┌────────────┐
q_0: ┤ X ├┤0           ├
     └───┘│            │
q_1: ─────┤  Measure_2 ├
          │            │
c_0: ═════╡0           ╞
          └────────────┘
c_1: ═══════════════════
                        


<Admonition type="attention" title="Important notes">

* There must be at least one classical register in order to use measurements.
* The Sampler primitive requires circuit measurements. You can add circuit measurements with the Estimator primitive, but they are ignored.

</Admonition>

## Next steps

<Admonition type="tip" title="Recommendations">
- [`Measure`](/docs/api/qiskit/circuit#qiskit.circuit.Measure) class
- [`measure_all`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) method
- [`measure_active`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#measure_active) method
- [`random_circuit`](/docs/api/qiskit/circuit_random#qiskit.circuit.random.random_circuit) method
</Admonition>